In [1]:
!pip install openai python-docx requests


In [2]:
import requests

# GSK OAuth URL and client credentials
token_url = "https://federation-qa.gsk.com/as/token.oauth2"
client_id = "eb98c3a954444f91a887dc054dd2037c"
client_secret = "XJJVX1bKrQ1YznWfzHunzVhTkQJP2I3wivNjBG2k2rJlHQNIDVV1AwL7TABtAZtcaMF4"

# Request token
response = requests.post(
    token_url,
    data={
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
)

# Parse token
access_token = response.json()["access_token"]
print("✅ Access token received")


✅ Access token received


In [3]:
# GSK Enterprise OpenAI Endpoint
endpoint = "https://dev.api.gsk.com/co/edap/openaidevtest02/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview"

# Headers with OAuth token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}


In [4]:
import json

def summarize_field(field, matched_paragraph):
    prompt = f"""
You are an AI assistant generating a summary for a clinical report.

Summarize the following paragraph as a proper response to the field '{field}'.

Paragraph:
{matched_paragraph}

Summary:
"""

    # Prepare payload
    payload = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.5
    }

    # Send request to GSK OpenAI endpoint
    response = requests.post(endpoint, headers=headers, data=json.dumps(payload))
    result = response.json()

    return result["choices"][0]["message"]["content"].strip()


In [5]:
import json

# Load matched_paragraphs from semantic_matching.json (from Day 2 output)
with open("matched_paragraphs.json", "r") as f:
    matched_paragraphs = json.load(f)


In [6]:
final_summaries = {}

for field, paragraph in matched_paragraphs.items():
    print(f"\n🟩 FIELD: {field}")
    print("🔍 Matched Paragraph:\n", paragraph)

    summary = summarize_field(field, paragraph)
    print("📝 Summary:\n", summary)

    final_summaries[field] = summary



🟩 FIELD: Protocol Summary - Master Disclosure Document for Interventional Studies
🔍 Matched Paragraph:
 6.	STUDY INTERVENTION(S) AND CONCOMITANT THERAPY	46
📝 Summary:
 **Protocol Summary - Master Disclosure Document for Interventional Studies:**

The study outlines the specific interventions administered to participants, detailing the primary investigational treatment(s) under evaluation. It also addresses guidelines for concomitant therapies, including permitted and prohibited medications or procedures that may be used alongside the study intervention. This section ensures standardized implementation of treatments and minimizes potential interference with study outcomes.

🟩 FIELD: The Information on this page will not be posted
🔍 Matched Paragraph:
 PAGE
📝 Summary:
 This page contains clinical information intended for internal use only and will not be publicly disclosed or posted.

🟩 FIELD: Before using this template for authoring, refer to the supplemental instructions on Find-IT. T